# Bond pricing models - zero-coupon, convertible bonds

## Import required libraries

In [93]:
import numpy as np
from scipy.stats import norm

## Zero-coupon bonds

We start with the simplest bond type, the zero-coupon bond, where there are no interest payments made, with the only cash flow occuring at the maturity of the bond when the bond-holder receives the face-value of the bond. 

Zero-coupon bond price formula:

$$P = \frac{FV}{(1 + r)^N}$$

Zero-coupon duration formula:

$$Duration = \frac{N}{1 + r}$$

Zero-coupon yield to maturity formula:

$$YTM = (\frac{FV}{P})*(\frac{1}{N}) - 1$$

Where:
* FV is the face value of the bond
* P is the price of the bond
* N is the maturity of the bond in years
* r is the yield to maturity in the decimal format

In [102]:
def zc_price(yield_to_maturity, maturity, face_value):
    price = face_value / (1 + yield_to_maturity)**maturity
    return price

In [103]:
def zc_duration(yield_to_maturity, maturity):
    bond_duration = maturity / (1 + yield_to_maturity)
    return bond_duration

In [110]:
def zc_yield_to_maturity(bond_price, maturity, face_value):
    yield_to_maturity = (face_value / bond_price)*(1 / maturity) - 1
    return yield_to_maturity

In [111]:
class ZeroCouponBond:
    def __init__(self, face_value, maturity, yield_to_maturity=None, market_price=None):
        self.face_value = face_value
        self.yield_to_maturity = yield_to_maturity
        self.maturity = maturity 
        self.market_price = market_price
        
    def price(self):
        if self.face_value is not None and self.yield_to_maturity is not None and self.maturity is not None:
            return zc_price(self.yield_to_maturity, self.maturity, self.face_value)
        else:
            print('Missing input(s) to calculate the zero coupon price.')
            return None
    
    def duration(self):
        if self.yield_to_maturity is not None and self.maturity is not None:
            return zc_duration(self.yield_to_maturity, self.maturity)
        else:
            print('Missing input(s) to calculate the zero coupon duration.')
            return None
        
    def ytm(self):
        if self.face_value is not None and self.price is not None and self.maturity is not None:
            return zc_yield_to_maturity(self.bond_price, self.maturity, self.face_value)
        else:
            print('Missing input(s) to calculate the zero coupon yield to maturity.')  
            return None

In [118]:
zc = ZeroCouponBond(1000, 10, 0.06, 600)

In [121]:
print(zc.price())

558.3947769151179


In [122]:
print(zc.duration())

9.433962264150942


In [123]:
print(zc.ytm())

-0.8333333333333333


## Convertible bonds

Convertible bonds are a specific type of bond where the bond can be converted to a specific number of shares at a predetermined price. There are multiple ways to value a conversion bond, which range in difficulty. The simplest way to value the convertible bond is to add together the independent value of the bond and the independent value of the call option.

$$Convertible Bond Price = Bond Price + Call Option Price$$

In [44]:
def bond_price(face_value, coupon_rate, discount_rate, maturity, freq):
    periods = maturity * freq 
    coupon_payment = (coupon_rate / freq) * face_value
    cash_flows = np.repeat(coupon_payment, periods) # Generates array of coupon payments for each period
    cash_flows[-1] += face_value # Adds face value payment to coupon_payments array
    
    periods = len(cash_flows) 
    
    # Creates an array which is a repeat of the first argument
    discount_factors = np.repeat(1 + (discount_rate/freq), periods)
    
    # Sets each argument in the array to the negative power of the period number
    discount_factors = np.power(discount_factors, -np.arange(1, periods + 1)) 
    
    # Multiplies the two corresponding values together
    return np.dot(cash_flows, discount_factors)

In [45]:
bond_price(1000, 0.05, 0.04, 10, 4)

1082.0867152848903

In [46]:
# Call price using Black-Scholes model
def call_price(C, S, r, maturity, sigma):
    d1 = (np.log(C / S) + (r + 0.5 * sigma**2) * maturity) / (sigma * np.sqrt(maturity))
    d2 = d1 - sigma * np.sqrt(maturity)
    price = C * norm.cdf(d1) - S * np.exp(-r * maturity) * norm.cdf(d2)
    return price

Where:
* C is the current stock price
* S is the strike price
* r is the risk-free rate
* t is the time to expiration or in this case the maturity
* sigma is the volatility

In [47]:
class ConvertibleBond:
    def __init__(self, face_value, coupon_rate, discount_rate, maturity, freq, C, S, r, sigma):
        self.face_value = face_value
        self.coupon_rate = coupon_rate
        self.discount_rate = discount_rate
        self.maturity = maturity
        self.freq = freq
        self.C = C
        self.S = S
        self.r = r
        self.sigma = sigma
    def price(self):
        price = bond_price(self.face_value, self.coupon_rate, self.discount_rate, self.maturity, self.freq) + call_price(self.C, self.S, self.r, self.maturity, self.sigma)
        return price

In [48]:
cb = ConvertibleBond(1000, 0.05, 0.04, 10, 4, 100, 105, 0.02, 0.2)

In [49]:
cb.price()

1112.8123616155947

It is worth noting that this convertible bond prices the option using the Black-Scholes method and while this can accurately price the stock alone, it does not consider the payment of dividends.

### Advantages and disadvantages of Convertible bonds

Advantages:
* Potential for higher returns because they can be converted into shares if the stock price rises.
* Downside protection as they offer a fixed income component, which provides a cushion against stock market losses.
* Lower interest rates
* Diversification as it offers investors a bond which has both debt and equity features.

Disadvantages:
* Lower yields - they offer the potential for higher returns, but they offer lower yields because of this conversion feature.
* Limited upside potential - they may not capture full upside potential because of the conversion ratio.
* Complex - so some investors won't understand them.
* Dilution - if they are redeemed for stock, they will dilute the shares of the existing shareholders.